In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 3
setup_one_gpu(GPU)

Picking GPU 3


In [2]:
import sys
import numpy as np
import os.path as osp
from tf_lab.iclr.evaluator import Evaluator
from general_tools.in_out.basics import files_in_subdirs
from tf_lab.iclr.helper import stored_synthetic_samples, top_evaluation_dir

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
class_name = 'chair'
do_mmd = True
save_res = False
top_out_dir = top_evaluation_dir

sample_mmd = False
n_samples = 5
ref_pop = 1000
sample_pop = 1000

mmd_loss = 'chamfer'
boost_samples = 1

special_tag = 'coverage_emd_boost_3'
skip = ['test', 'val']

In [5]:
evaluator = Evaluator(class_name)
evaluator.load_gt_data()

In [6]:
best_by_jsd = {'gmm': 'bneck_128_full_32_gaussians',
               'l_w_gan_large': 1800,
               'r_gan': 1700,
               'l_gan_chamfer': 300,
               'l_gan_emd': 100,
               'l_w_gan_small': 1800
              }

best_by_mmd = {'gmm': 'bneck_128_full_40_gaussians',               
               'l_w_gan_large': 2000,
               'r_gan': 1350,
               'l_gan_chamfer': 300,
               'l_gan_emd': 200,
               'l_w_gan_small': 1700
              }

In [7]:
exp_dict_reduced = stored_synthetic_samples(class_name).copy()
exp_dict_reduced.pop('l_w_gan_large', 'None')
# exp_dict_reduced.pop('gmm', 'None')
generators_zoo = exp_dict_reduced.keys()

In [8]:
epochs_to_eval = [1, 10, 100, 200, 400, 1000, 1500, 2000]
for experiment_name in generators_zoo:
    if 'gmm' in experiment_name: 
        continue

    print experiment_name
    
    top_sample_dir = stored_synthetic_samples(class_name)[experiment_name]
    if save_res:
        exp_name = osp.basename(top_sample_dir)
        f_mmd = open(osp.join(top_out_dir, class_name, 'mmd_' + special_tag + '_' + exp_name + '.txt'), 'w')
    else:            
        f_mmd = sys.stdout

    sample_files = []

    for epoch in epochs_to_eval:
        sample_files.append(osp.join(top_sample_dir, 'epoch_%d.npz' % (epoch,) ))
    
    for sample_file in sample_files:
        evaluator.prepare_sample_data(sample_file, boost_samples)
        sample_name = osp.basename(sample_file)[:-len('.npz')]
        print sample_name

        f_mmd.write(sample_name + '\n')

        evaluator.compute_coverage(loss=mmd_loss, sample_estimator=sample_mmd, n_samples=n_samples, 
                          ref_pop_size=ref_pop, sample_pop_size=sample_pop,
                          f_out=f_mmd, skip=skip)

        f_mmd.flush()
    print
    if save_res:
        f_mmd.close()

r_gan
epoch_1
epoch_1


KeyboardInterrupt: 

In [9]:
generators_zoo

['r_gan', 'gmm', 'l_gan_emd', 'l_w_gan_small', 'l_gan_chamfer']

In [10]:
for best_model, tag in zip([best_by_jsd, best_by_mmd], ['jsd', 'mmd']):
    if tag == 'mmd':
        continue
    print tag    
    for experiment_name in generators_zoo:
        print experiment_name
        top_sample_dir = stored_synthetic_samples(class_name)[experiment_name]        
        if save_res:
            exp_name = osp.basename(top_sample_dir)
            f_mmd = open(osp.join(top_out_dir, class_name, 'mmd_' + special_tag + '_'+ tag + '_' + exp_name + '.txt'), 'w')
        else:            
            f_mmd = sys.stdout

        sample_files = []

        if 'gan' in experiment_name:
            epoch = best_model[experiment_name]
            sample_files.append(osp.join(top_sample_dir, 'epoch_%d.npz' % (epoch,) ))
        elif 'gmm' in experiment_name:
            gaussian_type = best_model[experiment_name]
            sample_files.append(osp.join(top_sample_dir, gaussian_type + '.npz'))
        else:
            assert False
        
        for rep in range(1):
            for sample_file in sample_files:
                evaluator.prepare_sample_data(sample_file, boost_samples)
                sample_name = osp.basename(sample_file)[:-len('.npz')]
                print sample_name

            f_mmd.write(sample_name + '\n')

#             evaluator.compute_coverage(loss=mmd_loss, sample_estimator=sample_mmd, n_samples=n_samples, 
#                                   ref_pop_size=ref_pop, sample_pop_size=sample_pop,
#                                   f_out=f_mmd, skip=skip)

            evaluator.compute_mmd(loss=mmd_loss, sample_estimator=sample_mmd, n_samples=n_samples, 
                                  ref_pop_size=ref_pop, sample_pop_size=sample_pop,
                                  f_out=f_mmd, skip=skip)


            f_mmd.flush()
            print
            if save_res:
                f_mmd.close()

jsd
r_gan
epoch_1700
epoch_1700
train 0.00173016 0.00112358

gmm
bneck_128_full_32_gaussians
bneck_128_full_32_gaussians
train 0.00143499 0.000845851

l_gan_emd
epoch_100
epoch_100
train 0.00186199 0.00152819

l_w_gan_small
epoch_1800
epoch_1800
train 0.00152556 0.000927815

l_gan_chamfer
epoch_300
epoch_300
train 0.00155404 0.00109437

